In [108]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import numpy as np
import pandas as pd
import time
import re

In [2]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [109]:
class utilities:
    
    #def 
    
    def wait(self, seconds):

        s = np.random.normal(0, 1)
        time.sleep(seconds*abs(s))
    
    def get_element_by_path(self, driver, xpath):
        try:
            return driver.find_element_by_xpath(xpath)
        except Exception as e:
            print(e)
            return None

In [115]:
class IndeedItemParser:
    def __init__(self):
        self.driverPath = "C:\\Users\\Junior\\Documents\\selenium\\driver\\chromedriver.exe"
        
        
    def _get_title(self, driver):
        try:
            title = driver.find_element_by_xpath("//*[@class='jobsearch-DesktopStickyContainer']//h3")
        except Exception as e:
            print(e)
        return title.text
    
    def _get_name(self, driver):
        
        try:    
            name = driver.find_element_by_xpath("//*[contains(@class,'jobsearch-InlineCompanyRating')]//div[1]")
            return name.text
        except Exception as e:
            print(e)
        
    
    def _get_address(self,driver):
         try:   
            address = driver.find_element_by_xpath("//*[contains(@class,'jobsearch-InlineCompanyRating')]//div[3]")
            if address.text == "-":
                address = driver.find_element_by_xpath("//*[contains(@class,'jobsearch-InlineCompanyRating')]//div[4]")
            return address.text
         except Exception as e:
            address = driver.find_element_by_xpath("//span[@class='jobsearch-JobMetadataHeader-iconLabel'][1]")
            return address.text
    
    def _get_salaire(self,driver):
        #jobsearch-JobMetadataHeader-item 
        try:   
            address = driver.find_element_by_xpath("//*[contains(@class,'jobsearch-JobMetadataHeader-item')]//div[1]")
            outer = re.compile(r'[[S|s]alaire?[\s+]?:?[\s+]?(.*)e?[\s+]?\/(an|mois)|(.*)?[\s+]?par?[\s+]?mois')
            m = outer.search(address.text)
            if m is not None:
                return m.group(1)
            return []
        except Exception as e:
            return "-"
    
    def _get_description(self,driver):
        try:
            #jobDescriptionText
            e_description = driver.find_element_by_id("jobDescriptionText")
            return e_description.text
        except Exception as e:
            print(e)
    
    def _get_date(self,driver,url,name):
       # print(url)
        try:
            date_str = driver.find_element_by_xpath("//*[@class='jobsearch-JobMetadataFooter']")
            date_str_full = date_str.text
            date_tbl = date_str_full.split(" ")
            count_str = date_tbl[4]
            label = date_tbl[5]
            
            if name in date_str_full:
                date_str_full = date_str_full.replace(name, "")
                date_tbl = date_str_full.split(" ")
                count_str = date_tbl[5]
                label = date_tbl[6]
                
            if count_str == "a" :
                count_str = date_tbl[5]
                label = date_tbl[6]
                
            #print("date_str", date_tbl)
            date = datetime.now()
            
            if count_str == "30+":
                return date - timedelta(days=30)
            
            count = int(count_str)
            if "jour" in label:
                date = date - timedelta(days=count)
            elif "heur" in label:
                date = date - timedelta(hours=count)
            return date;
        except Exception as e:
            print(e)
    
    
    def parse(self,url):
        driver = webdriver.Chrome(self.driverPath)
        driver.get(url)
        driver.maximize_window()
        
        title = self._get_title(driver)
        name = self._get_name(driver)
        address = self._get_address(driver)
        date = self._get_date(driver, url,name)
        salaire = self._get_salaire(driver)
        description = self._get_description(driver)
        
        driver.close()
        
        return title, name, address, date, salaire, description

In [116]:
import os.path
from os import path
import json

class IndeedParserStateObject(object):
    def __init__(self, data):
        self.__dict__ = json.loads(data)
        
class IndeedParserStateHandler:
    
    def __init__(self):
        self.filename = "indeed.parser.state.json"
        self.object = self._load_file()
        print(type(self.object))
        #self.object = IndeedParserStateObject(self.data)
    
    def save_state(self, job, location, page_request="", page_item_index=-1):
        self.object["job"] = job
        self.object["location"] = location
        self.object["page_request"] = page_request
        self.object["page_item_index"]  = page_item_index
        #self.data  = json.dumps(self.object)
        
        #data = json.dumps(self.object)
        self._save_file(self.object)
        #self.object = IndeedParserStateObject(self.data)
        
    def is_current_job(self, job):
        if (self.object["job"] == "") | (self.object["job"] == job):
            return True
        return False
    
    def is_current_location(self, location):
        if (self.object["location"] == "") | (self.object["location"] == location):
            return True
        return False
    
    def is_current_page_request(self, page_request):
        if (self.object["page_request"] == "") | (self.object["page_request"] == page_request):
            return True
        return False
    
    def _save_file(self, data):
         with open(self.filename, 'w') as outfile:
                json.dump(data, outfile)
                
    def _load_file(self):
        
        if path.exists(self.filename) == True:
            with open(self.filename) as json_file:
                data = json.load(json_file)
        else:
            data = {
                     "job": "",
                     "location":"",
                     "page_request":"",
                     "page_item_index":-1,
                    }
            with open(self.filename, 'w') as outfile:
                json.dump(data, outfile)
        
        return data


In [112]:
from datetime import datetime, timedelta
import os.path

class IndeedPaser:
    def __init__(self):
        self.website = "https://www.indeed.fr"
        self.driverPath = "C:\\Users\\Junior\\Documents\\selenium\\driver\\chromedriver.exe"
        #self.dataset = pd.DataFrame(columns=['URL', 'Titre','Nom entreprise','Adresse','Date de publication', 'description'])
        
        if os.path.exists('indeed.csv') == True:
            self.dataset = pd.read_csv("indeed.csv")
        else:
            #URL,Titre,Nom entreprise,Adresse,Date de publication,salaire, description
            self.dataset = pd.DataFrame(columns=['URL','Titre','Nom entreprise','Adresse','Date de publication','salaire','description'])
        
        self.jobs = ["développeur", "data scientist", "data analyst", "business intelligence"]
        self.locations = ["Paris", "Lyon", "Toulouse", "Nantes", "Bordeaux"]
        self.utilities =  utilities()
        self.indeed_item_parser = IndeedItemParser()
        self.parser_state = IndeedParserStateHandler()
    
    def _get_pages_counts(self,driver):
        #searchCountPages
        searchCountPages_elt = driver.find_element_by_id("searchCountPages")
        searchCountPages = searchCountPages_elt.text.split()
        if len(searchCountPages) == 6:
            searchCountPages = int("{0}{1}".format(searchCountPages[3],searchCountPages[4])) 
        else :
            searchCountPages = searchCountPages[3]  
                    
        return (int)(searchCountPages / 18)
    
    def parse(self):
        browser = webdriver.Chrome(self.driverPath)
        browser.get(self.website)
        browser.maximize_window()
        
        for job in self.jobs:
            
            if self.parser_state.is_current_job(job) == False:
                continue
                    
            for location in self.locations:
                
                if self.parser_state.is_current_location(location) == False:
                    continue
                self.parser_state.save_state(job, location)
                
                query = "https://www.indeed.fr/jobs?q={0}&l={1}".format(job, location)
                browser.get(query)
                    
                pages_count = self._get_pages_counts(browser)
                
                for page_index in range(1, pages_count):
                    full_query = "{0}&start={1}".format(query,page_index)
                    
                    browser.get(full_query)
                    
                    items = browser.find_elements_by_xpath("//*[contains(@class,'clickcard')]")
                    dataset_len = len(self.dataset)
                    
                    for index_i,item in enumerate(items): 
                        title = item.find_element_by_xpath(".//*[contains(@class,'jobtitle')]")
                        item_link = title.get_attribute("href")
                        
                        try:
                            if (self.dataset["URL"].isin([item_link]).any()):
                                print(item_link)
                                print("existe déjà")
                            else:
                                print("existe pas")
                                title, name, address, date,salaire, description = self.indeed_item_parser.parse(item_link)
                                self.dataset.loc[dataset_len + index_i] = [item_link, title, name, address, date, salaire, description]
                                print("len(dataset)", len(self.dataset))
                        except Exception as e:
                            print(e)
                            #if (len(self.dataset) == 0) or (True not in self.dataset["URL"].isin([item_link])):
                        
                        #break
                    #break
                    self.dataset.to_csv("indeed.csv",index=False)
                    #https://www.indeed.fr/jobs?q=developpeur&l=paris&start=10
                    
                print(searchCountPages)
                #break
            #break    
            self.dataset.drop_duplicates(subset ="URL", keep = False, inplace = True) 

In [10]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install django

Note: you may need to restart the kernel to use updated packages.


In [104]:
class CsvToMongodbMigratorTool:
    
    def __init__(self):
        self.dataset = pd.read_csv("indeed.csv")
        self.mongodbdao = IndeedMongodbDao()
        
    def migrate(self):
        records = json.loads(self.dataset.T.to_json()).values()
        self.mongodbdao.insert_data_bulk(records)
        

In [75]:
from selenium import webdriver
import bs4
from bs4 import BeautifulSoup

data_set = pd.read_csv("indeed.csv")
data_set
#data_set
#for index, row in data_set.iterrows():
#    xtraSoup = BeautifulSoup(row["description"])
#    body = xtraSoup.find('body')
#    row["description"] = body.text
    
#data_set.to_csv("indeed.csv",index=False)
        
#data_set = data_set.drop(columns=['index'])
#data_set
#print(len(data_set))
#data_set.drop_duplicates(subset ="URL", keep = False, inplace = True) 
#print(len(data_set))
#data_set.to_csv("indeed.csv",index=False)
#record = data_set.iloc[1]

#indeeddao = IndeedMongodbDao()
#indeeddao.insert_data(record["URL"],record["Titre"], record["Nom entreprise"],record["Adresse"], record["Date de publication"], record["description"])


,URL,Titre,Nom entreprise,Adresse,Date de publication,description
0,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Développeur JAVA/J2EE,Activ Browser Technology,Boulogne-Billancourt (92),2019-10-01 15:41:50.559427,"Activ Browser, la créativité d’une Web Agency ..."
1,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Développeur Software 3D / Unity 3D / RM & RA /...,Theoris,75010 Paris 10e,2019-09-02 15:42:08.057680,Vous êtes passionné de Développement Software ...
2,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Développeur Ionic / Firebase,NEXTT,92200 Neuilly-sur-Seine,2019-09-18 15:42:22.787673,Dans le cadre du développement d'une applicati...
3,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Développeur Java H/F,Ratp Smart Systems,Paris (75),2019-09-09 15:42:39.629319,1er jet : Pour accélérer sa transformation et ...
4,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Développeur Android (H/F),Universign,Paris (75),2019-09-02 15:42:50.657003,Universign est le pure player de référence sur...
5,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Développeur Informatique H/F,VALENTE SECURYSTAR,94320 Thiais,2019-10-01 16:43:07.533091,"VALENTE SECURYSTAR, fabricant de portes blindé..."
6,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Développeur commercial,Denise omer design,Paris 8e (75),2019-09-08 15:45:33.822324,Dirigeante d’une agence d’architecture intérie...
7,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Data Engineer / Développeur NLP H/F,Wolters Kluwer,Paris (75),2019-09-02 15:45:51.241619,"Au sein du Pôle Droit et Réglementation, ratta..."
8,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Développeur Java H/F,Capgemini,Paris (75),2019-09-18 15:46:06.053725,Développeur JAVA JEE H/FCapgemini qu’est-ce qu...
9,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Développeur PHP / MySQL,PRIXAN,Paris (75),2019-09-02 15:46:19.657774,PRIXAN est un logiciel intelligent issu de 2 a...


In [76]:
y = 'https://www.indeed.fr/company/LEC-Conseil/jobs/D%C3%A9veloppeur-Fullstack-F-H-ad4f8380f4e66e4a?fccid=3d3e7ac112fae637&vjs=3'
if True not in data_set["URL"].isin([y]):
    print("yes")

In [67]:
data_set["URL"].iloc[10]

'https://www.indeed.fr/company/LEC-Conseil/jobs/D%C3%A9veloppeur-Fullstack-F-H-ad4f8380f4e66e4a?fccid=3d3e7ac112fae637&vjs=3'

In [82]:
#migratorTool = CsvToMongodbMigratorTool()
#migratorTool.migrate()

In [113]:
from pymongo import MongoClient 
from pymongo import errors 
from django.core.validators import URLValidator
from django.core.exceptions import ValidationError
class IndeedMongodbDao:
    def __init__(self):
        self.conn = MongoClient() 
        self.db = self.conn.Indeed
        self.collection = self.db.data
        self._create_index()
    
    def _create_index(self):
        index_name = 'url'
        if index_name not in self.collection.index_information():
            self.collection.create_index(index_name, unique=True)
    
    def _valid_url_format(self,url):
        val = URLValidator()
        try:
            val(url)
        except ValidationError as e:
            raise Exception('bad format for url {}'.format(ur))
    
    def insert_data_bulk(self,data):
        self.collection.insert_many(data)
    
    def insert_data(self, url, title, name, address, publication_date, description):
        
        try:
            if url == "":
                raise Exception('url cannot be empty {}'.format(ur))

            self._valid_url_format(url)

            if title == "":
                raise Exception('title cannot be empty {}'.format(title))

            if name == "":
                raise Exception('the name of company cannot be be empty {}'.format(title))

            if description == "":
                raise Exception('description of company cannot be be empty {}'.format(title))


            line_to_insert = {
                                "URL": url,
                                "title":title,
                                "name":name,
                                "adresse":address,
                                "publication_date":publication_date,
                                "description":description
                             }

            # Insert Data 
            result = self.collection.insert_one(line_to_insert) 
        except errors.DuplicateKeyError as e:
            print('DuplicateKeyError : url {0} aready exist'.format(url))
    
    def get_all_data(self):
        data = self.collection.find({})
        return data
    
    

In [117]:
item_parser = IndeedItemParser()
result = item_parser.parse("https://www.indeed.fr/voir-emploi?jk=9dc37ec01c9984dd&from=serp&vjs=3")
print(result)

('Développeur front-end', 'Be Free Talent', 'Paris (75)', datetime.datetime(2019, 9, 27, 0, 42, 36, 371506), '-', 'Emploi\nFormation : un bac + 2 dans une école d’informatique est largement suffisant pour accéder à ce métier ou une reconversion professionnelle via un centre de formation informatique (formation intensive de 3 à 6 mois) permet d’accéder au métier de développeur front-end ou consultant front-end. Et pour les plus malins, des cours d’openclassrooms sont largement suffisants.\nActivité : le développeur front-end travaille sur des projets de refonte et/ou création de site web.\nTrès souvent, il dispose des compétences en design qui le permet de réaliser des maquettes du futur site internet.\nIl doit disposer de solides compétences en HTML & CSS et jQuery.\nÉvolution : après quelques expériences professionnelles sur du front-end, le développeur front-end peut monter en compétence sur un langage de programmation back end tels que le PHP, Java et Ruby.…\nVous venez de finir vot

In [118]:
test = IndeedPaser()
test.parse()

<class 'dict'>
existe pas
len(dataset) 1
existe pas
len(dataset) 2
existe pas
len(dataset) 3
existe pas
len(dataset) 4
existe pas
len(dataset) 5
existe pas
len(dataset) 6
existe pas
len(dataset) 7
existe pas
len(dataset) 8
existe pas
len(dataset) 9
existe pas
len(dataset) 10
existe pas
len(dataset) 11
existe pas
len(dataset) 12
existe pas
len(dataset) 13
existe pas
len(dataset) 14
existe pas
len(dataset) 15
existe pas
len(dataset) 16
existe pas
len(dataset) 17
existe pas
len(dataset) 18
existe pas
len(dataset) 19
existe pas
len(dataset) 20
existe pas
len(dataset) 21
existe pas
len(dataset) 22
existe pas
len(dataset) 23
existe pas
len(dataset) 24
existe pas
len(dataset) 25
https://www.indeed.fr/rc/clk?jk=846f7f5da2e3fae6&fccid=1807e5727f702882&vjs=3
existe déjà
existe pas
len(dataset) 26
https://www.indeed.fr/rc/clk?jk=39c4bddff4ad36c6&fccid=c4fe03f05dd933f8&vjs=3
existe déjà
existe pas
len(dataset) 27
https://www.indeed.fr/rc/clk?jk=8ee618fc0da4253b&fccid=2c1ebe710125ee82&vjs=3
existe 

KeyboardInterrupt: 

In [7]:
test.dataset

,URL,Titre,Nom entreprise,Adresse,Date de publication,description
0,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Développeur JAVA/J2EE,Activ Browser Technology,Boulogne-Billancourt (92),2019-10-01 15:41:50.559427,"<p>Activ Browser, la créativité d’une Web Agen..."
1,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Développeur Software 3D / Unity 3D / RM & RA /...,Theoris,75010 Paris 10e,2019-09-02 15:42:08.057680,<p>Vous êtes passionné de <b>Développement Sof...
2,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Développeur Ionic / Firebase,NEXTT,92200 Neuilly-sur-Seine,2019-09-18 15:42:22.787673,<p>Dans le cadre du développement d'une applic...
3,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Développeur Java H/F,Ratp Smart Systems,Paris (75),2019-09-09 15:42:39.629319,<p><b>1er jet : </b></p><p>Pour accélérer sa t...
4,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Développeur Android (H/F),Universign,Paris (75),2019-09-02 15:42:50.657003,<div><div><b>Universign</b> est le pure player...
5,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Développeur Informatique H/F,VALENTE SECURYSTAR,94320 Thiais,2019-10-01 16:43:07.533091,"<p>VALENTE SECURYSTAR, fabricant de portes bli..."
6,https://www.indeed.fr/rc/clk?jk=846f7f5da2e3fa...,DÉVELOPPEUR(-EUSE) PRODUIT PAP (F/H) - CDD,Louis Vuitton,Paris (75),2019-09-19 15:43:18.703088,"<div><div><h2 class=""jobSectionHeader""><b>Post..."
7,https://www.indeed.fr/rc/clk?jk=b5268c024d6547...,Assistant développeur Front (H/F),OUI.sncf,Paris (75),2019-09-27 15:43:31.457057,<div><p>SIDH (Service d’Itinéraires et de Donn...
8,https://www.indeed.fr/company/CREATIVE-INGENIE...,Développeur PHP H/F,CREATIVE INGENIERIE,Paris 6e (75),2019-09-24 15:43:42.364257,<p><b>LES MISSIONS QUI TE SERONT CONFIEES : </...
9,https://www.indeed.fr/rc/clk?jk=a88bee6c673082...,Développeur iOS (H/F),Meetic Group,Paris (75),2019-10-02 05:43:55.761557,<div>Job Description<br>\n<br>\n<p>Derrière la...


In [106]:
import numpy as np 
class IndedRegexParser:
    
    def __init__(self, descriptif_text ):
        self.descriptif_text = descriptif_text
    
    def _get_items(self,pattern):   
        result_re = re.findall(pattern,self.descriptif_text)
        result = []
        for item_l_1 in result_re:
            for item_l_2 in item_l_1:
                if item_l_2 != "":
                    result.append(item_l_2)
        return result
    
    def _get_niveau_experience(self):
        #Niveau d'expérience : regex true
        result = self._get_items('([(e|E)\w+]xpérience?:?\s\d{1,15} ans?)|([(e|E)\w+]xpérience?\s?de\s?\d{1,15} ans?)|(\d{1,15} ans d\'[e|E]xpérience[s]?)')
        if len(result) == 0:
            outer = re.compile(r'Expérience:\s*.+:( (\w) (an|ans) \(((R|r)equis|(s|S)ouhaité)\))')
            m = outer.search(self.descriptif_text)
            if m is not None:
                return m.group(1)
            return []
    
    def _get_salaires(self):
        #Salaire ajusté en fonction du coût de la vie dans les différentes villes : regex true
        #return self._get_items('(\d{2,3}\s?(k|K)?)|(\d{2,3}\s?\d{3,5}\s?(k|K)?)|([(s|S)]alaire?[\s]?:?[\s]?\d{2,6}k?)|([(r|R)]émunération?[\s]?:?[\s]?\d{2,6}k?)|([(g|G)]ratification?[\s]?:?[\s]?\d{2,6}k?)') 
        #[[S|s]alaire?[\s+]?:?[\s+]?(.*)e?[\s+]?\/an
        outer = re.compile(r'[[S|s]alaire?[\s+]?:?[\s+]?(.*)e?[\s+]?\/(an|mois)|(.*)?[\s+]?par?[\s+]?mois')
        m = outer.search(self.descriptif_text)
        if m is not None:
            return m.group(1)
        return []
            
        
    
    def get_langages_prog(self):
        #Langages de programmation requis (R, Python, Excel (VBA), SQL, C++, Java, SAS...): regex true
        return self._get_items('(R+[^\w])|([(p|P)\w+]ython?)|(Excel?)|(VBA?)|(C\+\+?)||(C?)|(Asp.net?)|(SQL?)|(NoSQL?)|(Linux?)|(MySQL?)|(MongoDB?)|(DBMaria?)|(SQL Server?)||(Java?)|(JavaScript?)|(PHP?\s?\d{1,}?)|(HTML?\s?\d{1,}?)|(CSS?\s?\d{1,}?)|(SAS?)|(C#?)|(Ruby?)|(Swift?)|(Objective-C?)|(VB.NET?)|(Kotlin?)|(Scala?)|(Bash?)|(PowerShell?)|(Shell?)|(Front End?)|(Back End?)')
    
    def get_outils_re(self):
        #Outils requis (Tableau, PowerBI...) : regex true
        return self._get_items('([(t|T)\w+]ableau?)|([(p|P)\w+]ower[(b|B)\w+][(i|I)\w+]?)|([(s|S)\w+]ymfony?\s?\d{1,}?)|([(j|J)\w+][(q|Q)\w+]uery?\s?\d{1,}?)|([(a|A)\w+]ngular?[\s?\d{1,}]?)|([(r|R)\w+]eact [(j|J)\w+][(s|S)\w+]?)|(React Native?)|(Node JS?)|([(g|G)\w+]it?)|(Visual Studio?)|(Visual Studio Code?)|(Django?)|(Flask?)|(API REST?)|(http?s?)|(HTTP?S?)')
        
        
    def get_niv_etudes_re(self):
        #Niveau d'études requis : regex true
        return self._get_items('([(f|F)\w+]ormation?\s?[(b|B)\w+]ac\s\+\s[1-8])|([(b|B)\w+]ac\s\+\s[1-8])')
        
    def get_type_cursus_re(self):
        #Type de cursus (école ingénieur, master, autodidacte...) : regex true
        return self._get_items('([(é|E)\w+]cole [(i|I)\w+]ngénieur?)|([(m|M)\w+]aster?\s?\w{3,25})|([(a|A)\w+]utodidacte?)|([(g|G)\w+]rande[s]? [(é|E)\w+]cole[s]?)|([(é|E)\w+]cole[s]? de [(c|C)\w+]ommerce[s]?)|([(i|I)\w+]ngénieur [(i|I)\w+]nformatique?)')
        
    def get_type_contrat_re(self):
        #Type de contrat : regex true
        return self._get_items('[(c|C)\w+]ontrat?:?\s\w{3,25}')
    
    def get_categorie_developpeur_re(self):
        #Grande catégorie : développeur (developeur web, dev mobile,dev) / data regex true
        return self._get_items('([(d|D)\w+]éveloppeu[(r|s)]e? [(f|F)\w+]ull? [(s|S)\w+]tack)|([(d|D)\w+]éveloppeu[(r|s)]e? [(f|F)\w+]ront[(\s|\-)][(e|E)\w+]nd?)|(développeu[(r|s)]e? Back[(\s|\-)][(e|E)\w+]nd?)|([(d|D)\w+]éveloppeu[(r|s)]e? [(m|M)\w+]obile?)|([(d|D)\w+]éveloppeu[(r|s)]e? [(l|L)\w+]ogiciel?)|([(s|S)\w+]oftware [(i|I)\w+]ngenieer?)|([(d|D)\w+]éveloppeu[(r|s)]e? [(l|L)\w+]ogiciels?)|([(d|D)\w+]éveloppeu[(r|s)]e? [(C|c)\w+]\+\+?)|([(d|D)\w+]éveloppeu[(r|s)]e? [(c|C)\w+]#?)|([(d|D)\w+]éveloppeu[(r|s)]e? [(j|J)\w+]ava?)')
    
    def get_categorie_data_re(self):
        #Grande catégorie : data regex true
        return self._get_items('([(d|D)\w+]ata [(a|A)\w+]nalyst?)|([(d|D)\w+]ata [(s|S)\w+]cientist?)|([(i|I)\w+]ngénieu[(r|s)]e? [(m|M)\w+]achine [(i|I)\w+]earning?)|([(c|C)\w+]onsultant [(b|B)\w+]usiness [(i|I)\w+]ntelligence)|([(c|C)\w+]onsultant [(d|D)\w+]ata)|([(c|C)\w+]onsultant [(b|B)\w+]ig [(d|D)\w+]ata)|([(c|C)\w+]onsultant BI)')
        

In [93]:

dataset = pd.read_csv("indeed.csv")
print(len(dataset))
for index, row in dataset.head(1000).iterrows():
    print(row["URL"])
    indedRegexParser = IndedRegexParser(row["description"])
    #print(indedRegexParser._get_niveau_experience())
    print(indedRegexParser._get_salaires())
    #print(indedRegexParser.get_outils_re())
    #print(indedRegexParser.get_niv_etudes_re())
    #print(indedRegexParser.get_type_cursus_re())
    #print(indedRegexParser.get_type_contrat_re())
    #print(indedRegexParser.get_categorie_developpeur_re())
    #print(indedRegexParser.get_categorie_data_re())


1000
https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYlbfkN0BdiCki7slH8zOJUlzNTkvSdGq5XXDacoubTFlilpruQMpircP1oQQ5CFs4T2qu0AI_LQp7j1E-koadF_S6aKaubP2orTTt0lNwEg_T4qk6xbF4LDnDX7gfGNXc0okSEKVHYfpP8q1qMnrIVLDZ3uDdib30b8g3QdrdoBAiO_4ggKyxxOnwUsbTWocK38na_0bW-q_0-3g9v54jAJjsg_FlFXZlQkzNu16vVyNMDSkmBAkCTnoIS2jdUwMnxCZzQF1nFJpq0aaxTt5W0yFMFgOQ5ZMGDeDZ93hRfRwpwZhA1PMS9UpCkbWIhGx41UW-ZLR8_dG3wR2m4R4UYeojZZdcHsIHxdGXu-0pIefA6lT7CEPuo2AAVND-w2JY_ujTUfs4_V8J0Qff7O9zCHMF9H6VWfIsz1rVvnrX5nop-ZKge0G7kd4Z9_7N&p=0&fvj=1&vjs=3
33 000,00€ à 50 000,00€ /anCe poste est ouvert aux personnes en situation de handicap.Type d'emploi : CDISalaire : 35 000,00€ à 50 000,00€ 
https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYlbfkN0BPA0_wx9cDprZ7BJ5N77NUbc2ajC2I_x_zlGmtNnAwCfgmTUExHA28Yma9NzXrLsTN4stEHdEuocdTfP8hBmaakm9ZUCKPYcZAg0YExzTak9_N3-S77YYoZtvUW0y4EHqK1Kwq_IHMV2MKxNx2-t-vFGzsYTvzYwt6y5UIJJEAdl4smulknsMbk53xIFDBmKKoWawBCgxb3OqTnRUgEPYxwWpsBhGbYAd2kix18RagMVF9xcNKc2JPHTu-rcNMCwTivg5h5Qteg0-iNNja976hKLm_JYOwEtg42Q7k7Sc9

[]
https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYlbfkN0Dcb8hMDD42UCLCD3dd9omHjqRPm2HYHjhxQWztGxplNMjONztCM5eKZAxwe_vAPr12S1A6mLowwT9BU9u9PfcGXjWeGXp9GhG_l9vsKDVGgtUJDFqBvqjfwruuMxmdqXmGUOO2BhHuOnpANo7S6PD-xBVWfeGTDXlsKD5HcyiD3ea_BoVH1e9zxBJPNqHtbG9Xzfik0BxpzyunOLsdSJDtrN3C_2d7g_JFrAZDG3Sp51CXk9tgwhQ1yRvbiMPv-ntpH0Bcp4M_F6l3IxF0tlTJIUW7I0pjIOpNkMlM2M-MX7n_acliaKejXH12aYU3q9OTVXKLS8a4u1SX5TqakUsA22dHsdPFROAySFXJy303svAtj1OvYpyMmUy5moWX0yLmDMDPZNos1VPgG_1z2osBbYhEdxkqUwyNdLgHwUv6whrP7niW_dKsBJ_hgvIkFmYOaQB0gw2lwtNWbTsh6h-4&p=2&fvj=1&vjs=3
[]
https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYlbfkN0CiPpMlP4pYETjTx9_k_XKsM0xq2bc1vM3ECthSFUJs0nTOpAoQNRBYKcUiAbeGd6FLCuvzKH505B61gfT7C5_o1tGlPpb4Lbf5UEdBfBw_d0WU1pYiEaN_1Km2IHDBv65Ixd2bICzplYJQPsf7QPBnQ132GgpuIbS3HkLyY3gNAhYNg_ySQV1M4vjBmQHUEztJ-BBmO45SFmodc3nMHcYpeDdthXmbNCinelGjLF8_kk7BT-LpYCDIHUkirDo6R8WoGlGlFLdQjJvgGT-czhJLMpCng1DSpj5-HVhTskPkiXtPWypeqmnIYZWQwgnT78uDDPu6YOfzvm_LCE4n4brgU4FzfuNLUbTapTY40FqzQvWbvk_A7XrIeuhxq8pHwPs8RLL8kEtCHREnffDu28T

[]
https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYlbfkN0AtFAOVYquSy8vMzyjBWOxkfrpHwQATlMMyGSJSwptcoQMqDckLNDnfGHOMNvktQmnVB8-I_0gwiMZagjkkwJLvLfsrdzAsfJKKQJAyNRumXLgnRFyXw1hNLoDMWJVoG314jhcdLCElgY6wM500JFOIOh6CzUPJlHuFl_6ZH-rR-o6UvRWPzypBquzPCzP2kZVuS_jz37nRGQva0K73BPvdQVKvuwY970XS3_sI4LSlH-LpCLAjAubP8VWVaXGmumvJfSdL0-EZLzWJGAc8FvY4MYWSwSyFTGSafUj9W8zGAe8p3fhVxIVCZib8CD_-ly8KVd8nbnFMG3UqscEZjLpqthKgcaCSXF8bnpkvVf8pqT5Wni9i_as0yXmT6HdVbYTCJu7FieX_c7HQVTlI-T3OTpvr59adghChc70D7SawoDQ1HG-bf1x7Xa-Dz_H77xM=&p=5&fvj=1&vjs=3
1850,00€1 850€ + 10 Heures supplémentaire à 25% pendant la période d’essai (2 mois) soit un total de 2002 euro / mois.Type d'emploi : CDISalaire : 1 850,00€ 
https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYlbfkN0AMAWkgzOzSdb7VRV2y8otJLV6RKsPdFqKEK8w6T8Zc45JfcinlOj18r-eLvzAdXxXNpJj-PTEcjYHdkw-4MaW_r4j7-Q1mPCSFGyvy5Jc3-ncJ5jPobdX-PapOSWiU-C_aGO_6M6z1oFphThVk6AE1FiWs-qGuE7Ot7loGm9MhSDBBiIGCLk9NNll3WOxO98re-FFi49gftgBj2sdPrPQxtdnH8rnNjpl2c11NtoI30KKc1QqSbhZlVL9KFPcpMPD8nNwZJxbsMavs

45 000,00€ à 60 000,00€ 
https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYlbfkN0CvCzHKCV0pBpk3dBljYpIK7giJwAvpA3QpNppQJMstrYsXzreKNrmBQhi2Yt05rEiiz3bztyonHYJBPjcdA8vcukUCzS1zU_Gh-BM-rShw2AFX07o4bQpYDkydvzsUCk6nqP7ZI1sSYdTAixUk410wOm6GRWr8o26S3lZznhsVx9quW1Bu-ydZet8OuyApzXAwZH_Q0ChVpdtMapY2kaTHgVGiLubpRpc_Bi8t1d7RvC1Pr2zNKhoFPRjWBZK1SWSMqra3KBpG80vWg95OcDztbJDpjm9BvcUOxXeE0vzlqLZ3Fk83JjbRxOcHr04mnLF9YBDEbPWXeHbKVUZHkDPAST986rRIT_cShIu_cfEZTRTsuMDcT5gdZTDznVrLvo_54gS38ePBtP5BQYpkLqK36FeNQw8TivZt175C8Pi-UzVVBk1T5RKyYV2lYpYHRUaeHugARX1Q7g==&p=18&fvj=1&vjs=3
[]
https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYlbfkN0AtFAOVYquSy8vMzyjBWOxkfrpHwQATlMMyGSJSwptcoQMqDckLNDnfGHOMNvktQmnVB8-I_0gwiMZagjkkwJLvLfsrdzAsfJKKQJAyNRumXLgnRFyXw1hNLoDMWJVoG314jhcdLCElgY6wM500JFOIWgPVTtao4cU6qQfe-eSS2w-erqrKMEzAwRU8hNMz2hWmvprGflIU5HPF1xm48axqu6SRmy9kka6qHJqOBBOQu6IZimHVphTtAoLe-5opVQLYsh8AkqPCLnqD0fkTjmJphfW4qC-FW1RDgnekHW2mrtRx32G-nZ5YIEfRONOjniBJk1yk6oJJn3Fy42o64AEMU_kB1aSs5fy_VWT9W11OPX3CbRlAn776hnAWwLe47IAEXbVH

et les avantages40/45K€ selon expérience2 jours de télétravail par semaineMutuelle & TransportCe qu’on préfèreIls disposent de 4 bureaux : Nantes, Paris, Lyon et BordeauxParticiper à un projet qui est porteur de sens au regard de l’environnement et de la société civileEnvironnement technique moderneTrès bonne ambiance – confiance et travail en équipeSi vous êtes intéressé et/ou que vous un doute, postulez ! **Type d'emploi : CDISalaire : 40 000,00€ à 45 000,00€ 
https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYlbfkN0BPA0_wx9cDprZ7BJ5N77NUbc2ajC2I_x_zlGmtNnAwCfgmTUExHA28Yma9NzXrLsTN4stEHdEuocdTfP8hBmaakm9ZUCKPYcZAg0YExzTak9_N3-S77YYoZtvUW0y4EHqK1Kwq_IHMV2MKxNx2-t-vN12iC8iyyXtJqupNejm0dt3i1kVheeVMwUOjwfqysleEEbJWO_TQKGCshWS2DAbIWvNgyFHZpXBLXTlzxHQxp-w1AExsuY2xH_kw5ldZLCqemPodVtRxPMUj83_tCNwSojXYqk2Mf4EP3VWjd2DhQCSMOtOonus-9Ohfk8VN64nuUmPWQF-FAoJnQxyw29KNAlsPz7_HaksFukzqk-9SqlM4PqLb7CjAhfQSvQdezAliEtqSavuPtiKLFWBjOZG0NVO-j-JdfGQiIJC0YW7YCT6HmwSuiCx6DzHYt4rhysGNJDuHHjePVpHeEAVQT1pQRMZmoKON8bAA

...

[]
https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYlbfkN0CMZnBY86S_9WeJ_PZblJkeZGeUUI9AB8_gHvuA5S8H80AjvJU5FALdBvqB1-0L2ZE5lpxxDxOpT_U2oxSuNS0bfKr6VUm9VCGAIZQz6FxqABQH8YdKONRXtm2e6gdnAdYm3qX-SZ5uo9eit4Yf3PA9hJQ8VooRUK0g9EF8CekiscNVaW5BR6Ve3FZpSZ4hY7X3KtVEaWxflyYGI5A8Ms5unUfXQ3GyT-1FxFckB8bFXuUHWflXgTugFUA7HTufWJZJikDSpbV9lrma1otKO_nCwmKKJg2ID5cDRKh43NbLdzyMp8b_Kiw6JZ1PBLvaUcHLDfYs00lfSa4gZCwppixRqNs10k3FznYv7x5NvRU6w6zGMUTczsihiZbM-sCswIyGmvzmgaj9tXPIGrUD10PXHhGci-nawHPyCxSx4TE0HdkR1MlzgmFF&p=2&fvj=1&vjs=3
[]
https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYlbfkN0BR-D6Eajd0k4wLeJqANKltEnmoWZTg3-dYGMHnPq5F2GV-sj19SMZKQCmG5zXQHw6JFiP4_79KTvyZY9Ov5AHQsJ1zXTNEcYVnFotllfQJI9SiFEXXliwllp4Cx09cWoPWwZ-3urbJI4IPx5bYFPJGNNtC6tIjhGpEo0ab4IkVAvmBD6oLjNHnLn-1uxC-R4p1KrcEEkFMHVI--fAsLWZG-dZYJA_8uniHQ-pGikbIvVV1dDmRsOIoQScnWCCLWnHufEDn3nxcyXwEY3fZy7fiTy45RctPMKbpcB_-yySqnNtj6BjtQmrftc5eSi-yqadwHPl6XqeJ31gLuEIedUia2LvFS_59skc0SrgIHbP01lQNcuw7x94ACFebr-SDzqwk99ZqypkKb_A2S5tda1lFRt0VuiECNT1jWbkHZicH7oYKOaTSlcM

[]
https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYlbfkN0C-Gz5SMyttlQQQOJBNkHgv_xTmwZv36aH-8-sCJb1CoTkAtFCNkpsTUrS82lzXMX8Nfdhn8H4X_88um5ZGGEum5hO8O2GxSIFjAgZtIEnF9sHApvwMqys8dNZTlg-NavuWmuxnt4sihDFeit3LjBNk2gJH5HtY1dbs5tDrNwd_aZFW17KNomF2utJy4foIceF1uEsBKu-4bfh-cZFs0HEeVrfLeX6x2T_tPjybU_gGRcqNdF8p4FEMecE4KdDwZPqEGB4bcE6RwdbcyjZ1bl7ROwFHxnv_ahz6syc49fymkvCoOMViXv94HQ_e9LKAkC9btIP1gzYDL0L85VNsbp5jr4RHUOUDk5ZoS7u2vaT8MYi6jsAVtaO05xK6EpeeaASvN6elPkg3x_kEXpZhn3LZAJumGUrhvbZK_9RfNcSaMFOinnqRO0TlhL1Q1eSXFfFXi8nc4q77HnC7oCbvQYU3UPE4OgfppTA=&p=2&fvj=1&vjs=3
35 000,00€ à 80 000,00€ 
https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYlbfkN0Afb2xqmmvnLeU-eGdBKKKG4nWVW1VrA-McngqAoir6deY3vbRrCKR3mm24AFrxzAqh7I0b4RySssf7nCnyJKwdLxN7QMXQhkNsCK8XusozsvmW1eF3sglEZHhdIQHVv2RHiak76E1rzsanbeeCr-cVLuliJgda70mbwtUvljolTlJIx1Yf8D76yeLwPAtWEyilr35eu9duX_DPbd7FbsNGQw3q8KFCYRWi46_8NFXkDgWNNzqOJf6owN5alwXDFyRT67Ej7pJxEoyZMz5owj-rsp1ZFFV5qu5O1NuHfMjfV9ZK060eBzSO2CK1gtDee_WbqA0D5wjz7fYKnfe76F9X_YA6xaFTRmrwnd8op0o1DT5pE6eJ-

[]
https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYlbfkN0CMZnBY86S_9WeJ_PZblJkeZGeUUI9AB8_gHvuA5S8H80AjvJU5FALdBvqB1-0L2ZE5lpxxDxOpT_U2oxSuNS0bfKr6VUm9VCGAIZQz6FxqABQH8YdKONRXtm2e6gdnAdYm3qX-SZ5uo9eit4Yf3PA9KeIdW5Ifwp850GsrOQeedILgZTW_5uCuepAsO06nyhZ-wKkKyjNDVKbtTxCsiYW5cSBRNDyrsqVzQlnE_cONvJbpFByLF20mUuqi3Vf4-NmZoxVeHmL7sEBB6f_qyxxgDO7pm5MJw4PJVGbkclZne_8ODKCt2ToLu0XB4SkaaVmaQv6wE0jNjsXcL56ZONl8G79ISIJmuNAHcVAENMSpC5mV826CPdoGfDWY1AUxtitEOSDDaov1PjoFTNFHTcFMoDxd5EEJ1XNeHNTEDuLQoQ7-K4_QQle3&p=2&fvj=1&vjs=3
[]
https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYlbfkN0BdCqteZLp0lXVbomwX7iwWS2X7bnlHFWD5aYFK-F_Y50u8P1nWCutwbC-uUOgISsV8eiLIpde8TTlIrZhhn3uEEAfN0cy23l_lqALPXb1QggVtuZcj3lvBGHYPC7I3l1UTz-4BGubys_5D2v51m1dFY6odbL9GWDX5ibZLC5QPH9mdaJvTLxbacgCQ5FSgbsPlx_x6WcjPE2YmeuV5lK7kfflSGx9wTECSj07UpWLY-7lzDVF6XFx84t4KJ-VVympaNcn2d9mnPaWk6q6EM5OG_Al-zGOPG8N4p-Ucq0_-M07OVgbL5csAumNiyp5IYa5UCkVbJH3QMwHA0idSnL2sRiRAjIPAcn3FrvejzBcfwP5ZavJ9iDx-Bia03W-G4iM8vJjrfOY59vQl8sdt7hk3vizEcxkEhpq3HR_0PA5-NZw9YvlpENU

[]
https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYlbfkN0CMZnBY86S_9WeJ_PZblJkeZGeUUI9AB8_gHvuA5S8H80AjvJU5FALdBvqB1-0L2ZE5lpxxDxOpT_U2oxSuNS0bfKr6VUm9VCGAIZQz6FxqABQH8YdKONRXtm2e6gdnAdYm3qX-SZ5uo9eit4Yf3PA9fsMC3GH2RWmDkS2uBb5N1pa8pEhhy-6Flr7vsaKeZD0Jdmj11teaxiI6qVJahWaoNdgZ3V-ex6ug7lts-0GSCm1O60dc-3KgIc_uuGACy1xfGkqd7x-zRD1ddP0rB5VmGaXNRyTcUImjmMOYt1kfnax8XSQpgTdTInM2qkj8HNalBb8aw6SAozd6h5JdXqPhtZqOo8txXz-XErTZUTranYKXFIkrbl19JvNKMv6bmvV5Lwwvgn9QRy5Lehq4LrL6ISH4UtxyW3wfz-_tCQVQYk995XS3Yno0&p=2&fvj=1&vjs=3
[]
https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYlbfkN0CMZnBY86S_9WeJ_PZblJkeZGeUUI9AB8_gHvuA5S8H80AjvJU5FALdBvqB1-0L2ZE5lpxxDxOpT_U2oxSuNS0bfKr6VUm9VCGAIZQz6FxqABQH8YdKONRXtm2e6gdnAdYm3qX-SZ5uo9eit4Yf3PA9GfHegjBqHV4f9lBnFiqhcWsTC8D4EJrkxypkjmsXCta9VMXEVp9EK-wGCB7u95irzH2yOSNI3HIDdtWT2CSVqrgAJsOD2XBG6hy8jVZZKQ2yKUPvXmqRxIHlUpFoo-pqvmtK55_9inQ2w3pYntstwSCxkEkToUPpvziaGz0QKpPvBkW65FbTjrwkMmrB2TBJ2lEg9a75iZmIb1Hi9Rph6mteHLt2yQcc1mcv_xeeXA6QdhHcpc8KUgIQjZ3Gfi1vR4Q2hKfO1jzICGaniYhMkiaVPozCYwl

32 000,00€ à 55 000,00€ 
https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYlbfkN0BbuWnEmrif_ZtfGAzxATFIFXjE6NeLvdqEIffHh8KjAr3CgmnkvnVa45ZaGyYOkIPTYVno7tzXuk1P9A8e4qynDjtcbWFfCXk68Vn5W1c5GD39XWzSJAnN2MAru-43wSOcj-d3eBbwdwTGaZZU0ojXrOb_9X8oxqccOPnFBEGc2mT6lSod0qsor6WSQUtmt3EpJNB26Mk-YkpplDvYvwpZFu_Vhzj_276ZMO-iuSMdVby8AVkF92B-wpzn7ie0-Sjqswk0itijWaSay7u0gLRGGlekYee3TRiekjE3eP1zjdq2mFlfKj1xYXCgi3IaZP_S2NHSLrqYWdOiQD8Jfb9gzX0jIrXKHuOBqknTvjpRIB7ukodKfVLbPfMSH-TbupKmJvoRvlcZL1Wz5b0mIRQvLPBys-hjLzWkBoF4NoVDqcO9n2YbP4q-KD9PeTRw1Hzoo0BMOZO9Hg==&p=16&fvj=1&vjs=3
2 500,00€ à 3 000,00€ 
https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYlbfkN0AKwdcBbw7yaLakAlOy94N9O2J5L3XMJj_2kRwTNsXlUNYuX_gKE2_iolKcQN1B2oZIsU6uavldnGnoAM35u48XsGLFcky5H0PqBQ_mtGLQUrfP5ykEPWQrutmYexYiQQSSVeNu7LSeHrdhfnlBT2TVgd7B41XGnAcmJozDKMM2qLQLfPgK7xiKrubBe7HuGpC37pkNRpaFn9v037_GOS701Gx732Nl3x2677mPvISiQLRFf-8WIssgMtv2Va7Fq6o68RDk1JqMFS5NRQ8rZ7AaL3jIePWXfUOXRRiMjDmlTPrPO0c5YlIlocqJyPSQyEQhRf3DKvMMr6lvwz-uNqNYsvz_toGQjrxgQV3Ts2_wpo7g6-dC